In [1]:
import sys
import os
import pandas as pd

current_dir = os.path.dirname(os.path.abspath('__file__'))
project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"La root del progetto è: {project_root}")

La root del progetto è: c:\Users\David\Desktop\studio\bayes_naive\sms_spam


In [2]:
import pandas as pd

import io
from email import parser as email_parser
import re

from sklearn.model_selection import train_test_split


In [3]:
emailsSpam = pd.read_csv('./dati/email_origin.csv')

In [4]:
emailsSpam.head()

,label,origin
0,1,Return-Path: <RickyAmes@aol.com>\nReceived: fr...
1,0,Return-Path: <bounce-debian-mirrors=ktwarwic=s...
2,1,Return-Path: <7stocknews@tractionmarketing.com...
3,1,Return-Path: <vqucsmdfgvsg@ruraltek.com>\nRece...
4,1,Return-Path: <dcube@totalink.net>\nReceived: f...


In [5]:
emailsSpam.loc[0, 'origin']

'Return-Path: <RickyAmes@aol.com>\nReceived: from 129.97.78.23 ([211.202.101.74])\n\tby speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l38H7G0I003017;\n\tSun, 8 Apr 2007 13:07:21 -0400\nReceived: from 0.144.152.6 by 211.202.101.74; Sun, 08 Apr 2007 19:04:48 +0100\nMessage-ID: <WYADCKPDFWWTWTXNFVUE@yahoo.com>\nFrom: "Tomas Jacobs" <RickyAmes@aol.com>\nReply-To: "Tomas Jacobs" <RickyAmes@aol.com>\nTo: the00@speedy.uwaterloo.ca\nSubject: Generic Cialis, branded quality@ \nDate: Sun, 08 Apr 2007 21:00:48 +0300\nX-Mailer: Microsoft Outlook Express 6.00.2600.0000\nMIME-Version: 1.0\nContent-Type: multipart/alternative;\n\tboundary="--8896484051606557286"\nX-Priority: 3\nX-MSMail-Priority: Normal\nStatus: RO\nContent-Length: 988\nLines: 24\n\n----8896484051606557286\nContent-Type: text/html;\nContent-Transfer-Encoding: 7Bit\n\n<html>\n<body bgcolor="#ffffff">\n<div style="border-color: #00FFFF; border-right-width: 0px; border-bottom-width: 0px; margin-bottom: 0px;" align="center">\n<table s

In [6]:
emailsSpam.shape

(75419, 2)

In [7]:
def extract_raw_features(raw_email_text: str) -> dict:
    """
    Estrae le feature strutturali e il testo pulito dal contenuto RAW di un'email.
    """
    msg = email_parser.Parser().parse(io.StringIO(raw_email_text))
    
    features = {
        'subject': msg['subject'] or '',
        'from_header': msg['from'] or '',
        'content_type': msg.get_content_type(),
        'body_text': '',
        'is_html': False
    }

    body = ''
    if msg.is_multipart():
        for part in msg.walk():
            ctype = part.get_content_type()
            cdisp = part.get('Content-Disposition')

            if ctype == 'text/plain' and not cdisp:
                body += part.get_payload(decode=True).decode(errors='ignore')
            elif ctype == 'text/html':
                features['is_html'] = True
    else:
        body = msg.get_payload(decode=True).decode(errors='ignore')

    features['body_text'] = body
    
    from_domain_match = re.search(r'@([\w\.-]+)', features['from_header'])
    features['from_domain'] = from_domain_match.group(1).lower() if from_domain_match else 'unknown'
    
    features['body_length'] = len(body)
    
    return features

In [8]:
df_features = emailsSpam['origin'].apply(extract_raw_features)
df_parsed = df_features.apply(pd.Series)
df = pd.concat([emailsSpam, df_parsed], axis=1)

In [9]:
df.head()

,label,origin,subject,from_header,content_type,body_text,is_html,from_domain,body_length
0,1,Return-Path: <RickyAmes@aol.com>\nReceived: fr...,"Generic Cialis, branded quality@","""Tomas Jacobs"" <RickyAmes@aol.com>",multipart/alternative,,True,aol.com,0
1,0,Return-Path: <bounce-debian-mirrors=ktwarwic=s...,Typo in /debian/README,Yan Morin <yan.morin@savoirfairelinux.com>,text/plain,"Hi, i've just updated from the gulus and I che...",False,savoirfairelinux.com,730
2,1,Return-Path: <7stocknews@tractionmarketing.com...,authentic viagra,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...",multipart/alternative,Mega authenticV I A G R A $ DISCOUNT priceC...,True,tractionmarketing.com,130
3,1,Return-Path: <vqucsmdfgvsg@ruraltek.com>\nRece...,Nice talking with ya,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>",text/plain,"\nHey Billy, \n\nit was really fun going out t...",False,ruraltek.com,652
4,1,Return-Path: <dcube@totalink.net>\nReceived: f...,or trembling; stomach cramps; trouble in sleep...,"""Christi T. Jernigan"" <dcube@totalink.net>",multipart/related,"\nsystem"" of the home. It will have the capab...",True,totalink.net,2512


In [10]:
df.columns

Index(['label', 'origin', 'subject', 'from_header', 'content_type',
       'body_text', 'is_html', 'from_domain', 'body_length'],
      dtype='object')

In [11]:
df.loc[:, 'from_domain'].unique()

array(['aol.com', 'savoirfairelinux.com', 'tractionmarketing.com', ...,
       'gmcarsnorthwest.com', 'cdrcorp.com', 'noblecoffee.com'],
      shape=(25318,), dtype=object)

label: target -> devo farlo diventare la mia y
origin: mail originale -> rimuovere
subject: oggetto mail -> ('pulizia_punteggiatura', TextPreprocessor()) + ('tfidf', TfidfVectorizer(stop_words=stop_words_list))
from_header: chi ha inviato la mail -> probabilmente da rimuovere
content_type: tipo di contenuto -> one hot encoding
body_text: testo della mail -> ('pulizia_punteggiatura', TextPreprocessor()) + ('tfidf', TfidfVectorizer(stop_words=stop_words_list))
is_html: la mail è in html ? -> dummy variables
from_domain: dominio dal quale è arrivata la mail (con 25k domini) -> raggruppare mail in varie categorie
body_length: lunghezza del corpo della mail -> normalizzare (tra 0 e 1)

inizio splittando variabili dipendenti e indipendenti

In [12]:
X = emailsSpam['origin']
y = df['label']

In [13]:
X

0        Return-Path: <RickyAmes@aol.com>\nReceived: fr...
1        Return-Path: <bounce-debian-mirrors=ktwarwic=s...
2        Return-Path: <7stocknews@tractionmarketing.com...
3        Return-Path: <vqucsmdfgvsg@ruraltek.com>\nRece...
4        Return-Path: <dcube@totalink.net>\nReceived: f...
                               ...                        
75414    Return-Path: <Merrill8783@168city.com>\nReceiv...
75415    Return-Path: <Merrill8783@168city.com>\nReceiv...
75416    Return-Path: <r-help-bounces@stat.math.ethz.ch...
75417    Return-Path: <r-help-bounces@stat.math.ethz.ch...
75418    Return-Path: <the00@plg.uwaterloo.ca>\nReceive...
Name: origin, Length: 75419, dtype: object

In [14]:
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [15]:
y.value_counts()

label
1    50199
0    25220
Name: count, dtype: int64

ora eseguo il train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

In [17]:
print(X_train.shape)
print(X_test.shape)

(60335,)
(15084,)


In [18]:
X_train

23894    Return-Path: <darnell@grillini.com>\nReceived:...
49469    Return-Path: <dwdiamondcomputersm@diamondcompu...
56305    Return-Path: <barclaysbankb@aim.com>\nReceived...
65246    Return-Path: <r-help-bounces@stat.math.ethz.ch...
6013     Return-Path: <nobody@server.fastbox.info>\nRec...
                               ...                        
21243    Return-Path: <acm189991@hotmail.com>\nReceived...
45891    Return-Path: <jmandymoorethailand@goldenalaska...
42613    Return-Path: <eir@ins-th.com>\nReceived: from ...
43567    Return-Path: <samba-cvs-bounces+ktwarwic=speed...
68268    Return-Path: <contactus@activeroute.com>\nRece...
Name: origin, Length: 60335, dtype: object

## Piano per preprocessore
subject: oggetto mail -> ('pulizia_punteggiatura', TextPreprocessor()) + ('tfidf', TfidfVectorizer(stop_words=stop_words_list))
content_type: tipo di contenuto -> one hot encoding
body_text: testo della mail -> ('pulizia_punteggiatura', TextPreprocessor()) + ('tfidf', TfidfVectorizer(stop_words=stop_words_list))
is_html: la mail è in html ? -> dummy variables
from_domain: dominio dal quale è arrivata la mail (con 25k domini) -> raggruppare mail in varie categorie
body_length: lunghezza del corpo della mail -> normalizzare (tra 0 e 1)

## Ulteriori considerazioni
il dataset è sbilanciato con 2/3 degli elementi della classe 1 e solo 1/3 della classe 0
questo mi richiederà di sviluppare delle tecniche per ribilanciare

## pipelines necessarie

* pipeline di pulizia e tfidf da applicare a subject e body_text
* pipeline per ohe a content_type e is_html
* pipeline per il raggruppamento a from_domain
* pipeline per la normalizzazione da applicare a body_length
* pipeline finale che raggruppa tutto